In [3]:
## Necessary Packages

import numpy as np # package for playing with matrices


In [9]:
def normalize(raw):
    norm=[]
    for i in raw:
        if i<0:
            norm.append(-i/min(raw))
        elif i>0:
            norm.append(i/max(raw))
        else:
            norm.append(i)
    return norm
def lowpass_filter(YA,sample_range,filter_size): #YA data set, measured sample range, filter_size arbitrary frequency cutoff based on number of samples
                   
    r=YA.shape[0]                   
    s=sample_range #GHz
    z=np.fft.fftfreq(r)/(2*s)*r
    B=filter_size
    e=int((B)/z[1])
    t=np.linspace(0, s, r)

    fYA=np.fft.rfft(YA)
    filter_e=e
    fYAr=np.zeros(fYA.shape[0])
    fYAl=np.zeros(fYA.shape[0])
    fYAr[:filter_e]=np.real(fYA[:filter_e])
    fYAl[:filter_e]=np.imag(fYA[:filter_e])
    fYA_filt=np.array(fYAr)+np.array(1j*fYAl)
    ifY=np.fft.irfft(fYA_filt)
    return ifY

def outliers(value, L=5, U=95):
    """Check if value is an outlier based on percentiles
    """
    p25 = np.percentile(value, L)
    p75 = np.percentile(value, U)    
    
    lower = p25 - 1.5 * (abs(p75) - abs(p25))
    upper = p75 + 1.5 * (abs(p75) - abs(p25))
    
    return lower, upper

def get_outlier(value, L=5, U=95):
    """Check if value is an outlier based on percentiles
    """
    lower, upper = outliers(value, L, U)
    
    loutlier, uoutlier= value <= lower , value>= upper
    return loutlier, uoutlier # Give 2 output variables - e.g. lower , upper =is_outlier(data,5,95)

def get_outlier(value, L, U):
    """Check if value is an outlier based on percentiles
    """
    p25 = np.percentile(value, L)
    p75 = np.percentile(value, U)    
    
    lower = p25 - 1.5 * (p75 - p25)
    upper = p75 + 1.5 * (p75 - p25)
    loutlier, uoutlier= value <= lower , value>= upper
    return loutlier, uoutlier # Give 2 output variables - e.g. lower , upper =is_outlier(data,5,95)

def local_mean(matrix, y_axis, x_axis, size):
    """
    Outputs the local mean around the position  of size nearest neighbors disregarding the datapoint at [y_axis, x_axis]
    
    E.G. size =1 => mean of 8 (3^2-1) nearby cells (-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,-1)
         size =2 => mean of 24 (5^2-1) nearby cells )(-2,-2),(-2,-1)...(-2,1),(-2,2),(-1,-2)...(-1,2)...(2,-2)...(2,2)
         
         a = [[1,2,3,4,5],[6,7,8,9,10],[11,12,99,14,15],[16,17,18,19,20],[21,22,23,24,25]]
         local_mean(a,2,2,1) => (7+8+9+12+14+17+18+19)/8
         
         Can be easily chnaged to local_mean_inclusive that would include the point y_axis,x_axis
         
    """
    if type(matrix)==list:
        matrix=np.array(matrix)
    
    if matrix.shape[0]<3 or matrix.shape[1]<3:
        raise NameError('Matrix too small')
   
    if size==0:
        return matrix[y_axis,x_axis]
    
    local_data=[]
    numb=[]
    nsize=matrix.shape[1]-1
    msize=matrix.shape[0]-1
    
    if int(2*size+1)>int(matrix.shape[0]) or int(2*size+1)>int(matrix.shape[1]):
        
        size1=min(int(np.floor(matrix.shape[0]/2)) ,int(np.floor(matrix.shape[1])))
        size2=matrix.shape[0]
        size3=matrix.shape[1]
        if matrix.shape[0] % 2==0 :
            size2=int(matrix.shape[0]/2)-1
            
        if matrix.shape[1] % 2==0:
            size3=int(matrix.shape[1]/2)-1
        size=min(size1,size2,size3)
        


    for n in range(-size, size+1,1) :
        for m in range(-size, size+1,1):
            if m==0 and n==0:
                
                #local_data.append(np.nan)
                #numb.append(np.nan)
                continue
                
            if  abs(y_axis+n)>nsize and   abs(x_axis+m)>msize:  #Cycles around far corner back to 0,0 for arbitrary size
                k = (y_axis+n) 
                k = k % nsize
                l = (x_axis+m) 
                l = l % msize
                l=l-1
                k=k-1
                local_data.append(matrix[k, l])
                numb.append([k, l])
                #numb.append(99)
                continue
            
            if abs(y_axis+n)>nsize: # Cycles back around
                k = y_axis+n 
                k = k% nsize
                k=k-1
                local_data.append(matrix[k, x_axis + m])
                numb.append([k,x_axis + m])
                #numb.append(98)
                continue
            if   abs(x_axis+m)>msize: #Cycles back around
                l = (x_axis+m) 
                l = l % msize
                l=l-1
                local_data.append(matrix[y_axis +n, l])
                numb.append([y_axis+n, l])
                #numb.append(97)
                continue
          
            local_data.append(matrix[y_axis+n,x_axis+m])
            numb.append([y_axis+n,x_axis+m])
            
            
        
    loc_mean=sum(local_data)/((2*size+1)**2-1)        
        
    return loc_mean, local_data
  
def diff_mat(matrix):
    
    if type(matrix)==list:
        matrix=np.array(matrix)
    
    if matrix.ndim !=2:
        raise ValueError("input not 2 dim (n x m)")
    
    nsize=matrix.shape[0]
    msize=matrix.shape[1]
    diffy=[]
    diffx=[]
    for y in range(nsize):
        row_diff = matrix[y,:]-matrix[y-1,:]
        diffy.append(row_diff)
    for x in range(msize):
        col_diff = matrix[:,x]-matrix[:,x-1]
        diffx.append(col_diff)
    diffy = np.array(diffy)
    diffx = np.transpose(np.array(diffx)) 
    
    return diffy, diffx

def diff_outliers_bool(matrix, L=5, U=95):
    
    if matrix.ndim !=2:
        raise ValueError("input not 2 dim (n x m)")
    
    diffy, diffx = diff_mat(matrix)
    
    diffy_out, diffx_out = get_outlier(diffy, L, U), get_outlier(diffx, L, U)
    diffy_bool, diffx_bool = np.logical_and(diffy_out, diffx_out)
    diff_outliers = np.logical_or(diffy_bool, diffx_bool)
    
    return diff_outliers

def diff_outliers_index(matrix, L=5, U=95):
    
    if matrix.ndim !=2:
        raise ValueError("input not 2 dim (n x m)")
    
    diff_outliers = diff_outliers_bool(matrix)
    
    diff_outliers_index = np.transpose(np.where(diff_outliers))
    
    return diff_outliers_index


In [11]:
a=[[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15],[16,17,18,19,20],[21,22,23,24,25]]
#a=np.random.rand(4,7)
#a=[[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]]
b=[[1,2,3],[4,5,6],[7,8,9]]
diff_mat(b)

(array([[-6, -6, -6],
        [ 3,  3,  3],
        [ 3,  3,  3]]), array([[-2,  1,  1],
        [-2,  1,  1],
        [-2,  1,  1]]))

In [293]:
a=np.array(a)
a[0,2]

3